# Fama-French Risk Factors SML & HML
by Dr Liang Jin

Part of Mini Python Sessions: [github.com/drliangjin/minipy](https://github.com/drliangjin/minipy)

Based on the Python example on WRDS by Qingyi Song Drechsler: [Fama-French Factors (Python)](https://wrds-www.wharton.upenn.edu/pages/support/applications/risk-factors-and-industry-benchmarks/fama-french-factors-python/)

### Import external packages

In [1]:
# import pacakges
import pandas as pd
import numpy as np
import datetime as dt
import wrds
#import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

### Connect to WRDS

In [63]:
# WRDS Connection
conn = wrds.Connection()

Enter your WRDS username [liang]:jinlums
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


### General Settings

In [3]:
# pre-define the sample periods
COMPUSTAT_BEG_DATE = '01/01/1959'
CRSP_BEG_DATE = '01/01/1959'
CRSP_END_DATE = '12/31/2017'

### Retrieving Compustat Data

In [4]:
# Postgres Query
stmt = """
          SELECT gvkey, datadate, at, pstkl, txditc, pstkrv, seq, pstk
          FROM comp.funda
          WHERE indfmt='INDL' 
          AND datafmt='STD'
          AND popsrc='D'
          AND consol='C'
          AND datadate >= '{}'
       """.format(COMPUSTAT_BEG_DATE)
comp = conn.raw_sql(stmt)

### Work on Compustat Data

In [6]:
# set date and time to the standard format recognised by Pandas and other packages
comp['datadate']=pd.to_datetime(comp['datadate'])

# create a new variable for year
comp['year']=comp['datadate'].dt.year

In [7]:
# check data
comp.info()
comp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499800 entries, 0 to 499799
Data columns (total 9 columns):
gvkey       499800 non-null object
datadate    499800 non-null datetime64[ns]
at          431177 non-null float64
pstkl       429857 non-null float64
txditc      395617 non-null float64
pstkrv      427630 non-null float64
seq         419769 non-null float64
pstk        427502 non-null float64
year        499800 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 34.3+ MB


,gvkey,datadate,at,pstkl,txditc,pstkrv,seq,pstk,year
0,001000,1961-12-31,NaN,0.0,0.000,NaN,NaN,NaN,1961
1,001000,1962-12-31,NaN,0.0,NaN,NaN,NaN,0.0,1962
2,001000,1963-12-31,NaN,0.0,0.008,0.0,0.553,0.0,1963
3,001000,1964-12-31,1.416,0.0,0.020,0.0,0.607,0.0,1964
4,001000,1965-12-31,2.310,0.0,0.000,0.0,0.491,0.0,1965


### Deal with prefered stock

In [8]:
# if pstkrv is missing, then use pstkl ==>
comp['ps'] = np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])

# if created ps is missing, then use pstk ==>
comp['ps'] = np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])

# if ps is still missing, then assign 0
comp['ps'] = np.where(comp['ps'].isnull(),0,comp['ps'])

In [9]:
# again check prefered stock we just created
comp['ps'].describe()

count    499800.000000
mean         25.043483
std        1075.263139
min        -101.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      526300.000000
Name: ps, dtype: float64

### Book Value of Equity

In [10]:
# assign 0 to txditc ==>
comp['txditc'] = comp['txditc'].fillna(0)

# create a variable, be, for book value of equity ... broadcasting
comp['be'] = comp['seq'] + comp['txditc'] - comp['ps']

# if be is negative, replaced by NaN 
comp['be'] = np.where(comp['be'] > 0, comp['be'], np.nan)

In [11]:
# check book value of equity
comp['be'].describe()

count    378367.000000
mean       1013.919581
std        6170.248438
min           0.001000
25%          10.298500
50%          52.390000
75%         284.706000
max      404478.000000
Name: be, dtype: float64

### House cleanning

In [12]:
# sort values so that the dataframe is constructed by id and time
comp = comp.sort_values(by=['gvkey','datadate']) # gvkey is the primary company id

# count obs? starting from 0 to length of the group -1
comp['count'] = comp.groupby(['gvkey']).cumcount()

# house cleanning
comp = comp[['gvkey','datadate','year','be','count']]

In [13]:
comp.info()
comp.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499800 entries, 0 to 499799
Data columns (total 5 columns):
gvkey       499800 non-null object
datadate    499800 non-null datetime64[ns]
year        499800 non-null int64
be          378367 non-null float64
count       499800 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 22.9+ MB


,gvkey,datadate,year,be,count
0,001000,1961-12-31,1961,NaN,0
1,001000,1962-12-31,1962,NaN,1
2,001000,1963-12-31,1963,0.561,2
3,001000,1964-12-31,1964,0.627,3
4,001000,1965-12-31,1965,0.491,4
5,001000,1966-12-31,1966,0.834,5
6,001000,1967-12-31,1967,0.744,6
7,001000,1968-12-31,1968,2.571,7
8,001000,1969-12-31,1969,10.211,8
9,001000,1970-12-31,1970,10.544,9


### Retrieving CRSP stock data

In [14]:
# sql for returning a merged crsp price dataset
# its a large dataset, takes time to run
stmt = """
          SELECT a.permno, a.permco, a.date, a.ret, a.retx, a.shrout, a.prc,
                 b.shrcd, b.exchcd
          FROM crsp.msf AS a
          LEFT JOIN crsp.msenames AS b
          ON a.permno=b.permno
          AND b.namedt<=a.date
          AND a.date<=b.nameendt
          WHERE a.date BETWEEN '{}' AND '{}'
          AND b.exchcd BETWEEN 1 AND 3
       """.format(CRSP_BEG_DATE, CRSP_END_DATE)
crsp_m = conn.raw_sql(stmt)

In [15]:
# check data
crsp_m.info()
crsp_m.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3838338 entries, 0 to 3838337
Data columns (total 9 columns):
permno    float64
permco    float64
date      object
ret       float64
retx      float64
shrout    float64
prc       float64
shrcd     float64
exchcd    float64
dtypes: float64(8), object(1)
memory usage: 263.6+ MB


,permno,permco,date,ret,retx,shrout,prc,shrcd,exchcd
0,10000.0,7952.0,1986-01-31,NaN,NaN,3680.0,-4.375000,10.0,3.0
1,10000.0,7952.0,1986-02-28,-0.257143,-0.257143,3680.0,-3.250000,10.0,3.0
2,10000.0,7952.0,1986-03-31,0.365385,0.365385,3680.0,-4.437500,10.0,3.0
3,10000.0,7952.0,1986-04-30,-0.098592,-0.098592,3793.0,-4.000000,10.0,3.0
4,10000.0,7952.0,1986-05-30,-0.222656,-0.222656,3793.0,-3.109375,10.0,3.0


### Data cleanning

In [17]:
# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']] = crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# personally, I like to set all the identifications to be strings...because of the annoying trailing zeros

In [18]:
# format datatime
crsp_m['date'] = pd.to_datetime(crsp_m['date'])

# MonthEnd is a function from pandas.tseries.offsets
# convert timestamp to current month end <= for easier merging purpose
# MonthEnd(-1) move backwards by 1 month (last month end)
# MonthEnd(1) next month end = MonthEnd(0)
crsp_m['jdate'] = crsp_m['date'] + MonthEnd(0) # jdate stands for "join date", a consistent datetime to merge data

### Add delisting return

In [19]:
# again, sql query
dlret = conn.raw_sql("SELECT permno, dlret, dlstdt FROM crsp.msedelist")

# work on datetime
dlret['dlstdt'] = pd.to_datetime(dlret['dlstdt'])
dlret['jdate'] = dlret['dlstdt'] + MonthEnd(0)

In [20]:
# merge two datasets
crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])

### House cleaning

In [21]:
# house cleaning
crsp['dlret'] = crsp['dlret'].fillna(0)
crsp['ret'] = crsp['ret'].fillna(0)
crsp['retadj'] = (1 + crsp['ret']) * (1 + crsp['dlret']) - 1
crsp['me'] = crsp['prc'].abs() * crsp['shrout'] # calculate market equity
crsp = crsp.drop(['dlret','dlstdt','prc','shrout'], axis = 1)
crsp = crsp.sort_values(by = ['jdate','permco','me'])

In [22]:
crsp.info()
crsp.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3838338 entries, 505102 to 3569983
Data columns (total 10 columns):
permno    int64
permco    int64
date      datetime64[ns]
ret       float64
retx      float64
shrcd     int64
exchcd    int64
jdate     datetime64[ns]
retadj    float64
me        float64
dtypes: datetime64[ns](2), float64(4), int64(4)
memory usage: 322.1 MB


,permno,permco,date,ret,retx,shrcd,exchcd,jdate,retadj,me
505102,17670,74,1959-01-30,0.016667,0.006410,10,1,1959-01-31,0.016667,17034.5
541198,18702,267,1959-01-30,0.149254,0.149254,10,1,1959-01-31,0.149254,28297.5
604317,20714,584,1959-01-30,0.129412,0.129412,10,1,1959-01-31,0.129412,40512.0
156117,11287,921,1959-01-30,0.097403,0.097403,10,1,1959-01-31,0.097403,37180.0
617212,21151,994,1959-01-30,0.009009,0.009009,10,1,1959-01-31,0.009009,14826.0


### Aggregate market-cap to company level

In [23]:
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
# permno with largest mktcap in a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()

# join by jdate/maxme to find the permno
crsp1 = pd.merge(crsp, crsp_maxme, how = 'inner', on = ['jdate','permco','me'])
# drop me column and replace with the sum me
crsp1 = crsp1.drop(['me'], axis = 1)
# join with sum of me to get the correct market cap info
crsp2 = pd.merge(crsp1, crsp_summe, how = 'inner', on = ['jdate','permco'])
# sort by permno and date and also drop duplicates
crsp2 = crsp2.sort_values(by = ['permno','jdate']).drop_duplicates()

In [24]:
# inspect databases
crsp2.info()
crsp2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3764430 entries, 1176176 to 3762625
Data columns (total 10 columns):
permno    int64
permco    int64
date      datetime64[ns]
ret       float64
retx      float64
shrcd     int64
exchcd    int64
jdate     datetime64[ns]
retadj    float64
me        float64
dtypes: datetime64[ns](2), float64(4), int64(4)
memory usage: 315.9 MB


,permno,permco,date,ret,retx,shrcd,exchcd,jdate,retadj,me
1176176,10000,7952,1986-01-31,0.000000,NaN,10,3,1986-01-31,0.000000,16100.000000
1182378,10000,7952,1986-02-28,-0.257143,-0.257143,10,3,1986-02-28,-0.257143,11960.000000
1188589,10000,7952,1986-03-31,0.365385,0.365385,10,3,1986-03-31,0.365385,16330.000000
1194810,10000,7952,1986-04-30,-0.098592,-0.098592,10,3,1986-04-30,-0.098592,15172.000000
1201041,10000,7952,1986-05-30,-0.222656,-0.222656,10,3,1986-05-31,-0.222656,11793.859375


### Work on FF datetime

In [25]:
# create year and month
crsp2['year'] = crsp2['jdate'].dt.year
crsp2['month'] = crsp2['jdate'].dt.month
# keep December market cap
decme = crsp2[crsp2['month'] == 12]
# house keeping
decme = decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})

In [26]:
decme.info()
decme.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317351 entries, 1246000 to 3762625
Data columns (total 5 columns):
permno    317351 non-null int64
date      317351 non-null datetime64[ns]
jdate     317351 non-null datetime64[ns]
dec_me    317351 non-null float64
year      317351 non-null int64
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 14.5 MB


,permno,date,jdate,dec_me,year
1246000,10000,1986-12-31,1986-12-31,1981.546875,1986
1246001,10001,1986-12-31,1986-12-31,6937.000000,1986
1328049,10001,1987-12-31,1987-12-31,5828.000000,1987
1411147,10001,1988-12-30,1988-12-31,6362.250000,1988
1491927,10001,1989-12-29,1989-12-31,10347.750000,1989


In [27]:
### July to June dates
crsp2['ffdate'] = crsp2['jdate'] + MonthEnd(-6)
crsp2['ffyear'] = crsp2['ffdate'].dt.year
crsp2['ffmonth'] = crsp2['ffdate'].dt.month
crsp2['1+retx'] = 1 + crsp2['retx']
crsp2 = crsp2.sort_values(by = ['permno','date'])

In [28]:
# inspect
crsp2.info()
crsp2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3764430 entries, 1176176 to 3762625
Data columns (total 16 columns):
permno     int64
permco     int64
date       datetime64[ns]
ret        float64
retx       float64
shrcd      int64
exchcd     int64
jdate      datetime64[ns]
retadj     float64
me         float64
year       int64
month      int64
ffdate     datetime64[ns]
ffyear     int64
ffmonth    int64
1+retx     float64
dtypes: datetime64[ns](3), float64(5), int64(8)
memory usage: 488.2 MB


,permno,permco,date,ret,retx,shrcd,exchcd,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx
1176176,10000,7952,1986-01-31,0.000000,NaN,10,3,1986-01-31,0.000000,16100.000000,1986,1,1985-07-31,1985,7,NaN
1182378,10000,7952,1986-02-28,-0.257143,-0.257143,10,3,1986-02-28,-0.257143,11960.000000,1986,2,1985-08-31,1985,8,0.742857
1188589,10000,7952,1986-03-31,0.365385,0.365385,10,3,1986-03-31,0.365385,16330.000000,1986,3,1985-09-30,1985,9,1.365385
1194810,10000,7952,1986-04-30,-0.098592,-0.098592,10,3,1986-04-30,-0.098592,15172.000000,1986,4,1985-10-31,1985,10,0.901408
1201041,10000,7952,1986-05-30,-0.222656,-0.222656,10,3,1986-05-31,-0.222656,11793.859375,1986,5,1985-11-30,1985,11,0.777344


### Stock level characteristics

In [29]:
# cumret by stock
crsp2['cumretx'] = crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()
# lag cumret
# should we sort data first? or otherwise we are getting wrong dates?
crsp2['lcumretx'] = crsp2.groupby(['permno'])['cumretx'].shift(1) 

In [30]:
# inspect
crsp2.info()
crsp2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3764430 entries, 1176176 to 3762625
Data columns (total 18 columns):
permno      int64
permco      int64
date        datetime64[ns]
ret         float64
retx        float64
shrcd       int64
exchcd      int64
jdate       datetime64[ns]
retadj      float64
me          float64
year        int64
month       int64
ffdate      datetime64[ns]
ffyear      int64
ffmonth     int64
1+retx      float64
cumretx     float64
lcumretx    float64
dtypes: datetime64[ns](3), float64(7), int64(8)
memory usage: 545.7 MB


,permno,permco,date,ret,retx,shrcd,exchcd,jdate,retadj,me,year,month,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx
1176176,10000,7952,1986-01-31,0.000000,NaN,10,3,1986-01-31,0.000000,16100.000000,1986,1,1985-07-31,1985,7,NaN,NaN,NaN
1182378,10000,7952,1986-02-28,-0.257143,-0.257143,10,3,1986-02-28,-0.257143,11960.000000,1986,2,1985-08-31,1985,8,0.742857,0.742857,NaN
1188589,10000,7952,1986-03-31,0.365385,0.365385,10,3,1986-03-31,0.365385,16330.000000,1986,3,1985-09-30,1985,9,1.365385,1.014286,0.742857
1194810,10000,7952,1986-04-30,-0.098592,-0.098592,10,3,1986-04-30,-0.098592,15172.000000,1986,4,1985-10-31,1985,10,0.901408,0.914286,1.014286
1201041,10000,7952,1986-05-30,-0.222656,-0.222656,10,3,1986-05-31,-0.222656,11793.859375,1986,5,1985-11-30,1985,11,0.777344,0.710714,0.914286


### Deal with lag market Cap, properly

In [31]:
# lag market cap
crsp2['lme'] = crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count'] = crsp2.groupby(['permno']).cumcount()
crsp2['lme'] = np.where(crsp2['count'] == 0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

In [32]:
# baseline me (june market cap?)
mebase = crsp2[crsp2['ffmonth'] == 1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

In [33]:
mebase.info()
mebase.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314146 entries, 1213671 to 3735804
Data columns (total 3 columns):
permno    314146 non-null int64
ffyear    314146 non-null int64
mebase    311048 non-null float64
dtypes: float64(1), int64(2)
memory usage: 9.6 MB


,permno,ffyear,mebase
1213671,10000,1986,11734.59375
1213672,10001,1986,6033.12500
1292965,10001,1987,5822.12500
1376735,10001,1988,6200.00000
1458522,10001,1989,7007.00000


In [34]:
# merge result back together
crsp3 = pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])

In [35]:
crsp3.info()
crsp3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3764430 entries, 0 to 3764429
Data columns (total 21 columns):
permno      int64
permco      int64
date        datetime64[ns]
ret         float64
retx        float64
shrcd       int64
exchcd      int64
jdate       datetime64[ns]
retadj      float64
me          float64
year        int64
month       int64
ffdate      datetime64[ns]
ffyear      int64
ffmonth     int64
1+retx      float64
cumretx     float64
lcumretx    float64
lme         float64
count       int64
mebase      float64
dtypes: datetime64[ns](3), float64(9), int64(9)
memory usage: 631.8 MB


,permno,permco,date,ret,retx,shrcd,exchcd,jdate,retadj,me,...,month,ffdate,ffyear,ffmonth,1+retx,cumretx,lcumretx,lme,count,mebase
0,10000,7952,1986-01-31,0.000000,NaN,10,3,1986-01-31,0.000000,16100.000000,...,1,1985-07-31,1985,7,NaN,NaN,NaN,NaN,0,NaN
1,10000,7952,1986-02-28,-0.257143,-0.257143,10,3,1986-02-28,-0.257143,11960.000000,...,2,1985-08-31,1985,8,0.742857,0.742857,NaN,16100.0,1,NaN
2,10000,7952,1986-03-31,0.365385,0.365385,10,3,1986-03-31,0.365385,16330.000000,...,3,1985-09-30,1985,9,1.365385,1.014286,0.742857,11960.0,2,NaN
3,10000,7952,1986-04-30,-0.098592,-0.098592,10,3,1986-04-30,-0.098592,15172.000000,...,4,1985-10-31,1985,10,0.901408,0.914286,1.014286,16330.0,3,NaN
4,10000,7952,1986-05-30,-0.222656,-0.222656,10,3,1986-05-31,-0.222656,11793.859375,...,5,1985-11-30,1985,11,0.777344,0.710714,0.914286,15172.0,4,NaN


In [36]:
# create a new variable from lag market cap (for weight later on)
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

decme['year'] = decme['year'] + 1
decme = decme[['permno','year','dec_me']]

# Info as of June so that we have data in one place (row) to construct our portfolios
crsp3_jun = crsp3[crsp3['month'] == 6]

crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])

### House cleanning

In [37]:
# make our crsp table prettier...
crsp_jun = crsp_jun[['permno','date', 'jdate', 'shrcd','exchcd','retadj','me','wt','cumretx','mebase','lme','dec_me']]
crsp_jun = crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()

### Retrieving CCM data

In [38]:
ccm=conn.raw_sql("""
                    SELECT gvkey, lpermno AS permno, linktype, linkprim, 
                    linkdt, linkenddt
                    FROM crsp.ccmxpf_linktable
                    WHERE SUBSTR(linktype,1,1)='L'
                    AND (linkprim ='C' OR linkprim='P')
                 """)

In [39]:
# convert datetime
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

### Merge with Compustat and CRSP

In [40]:
# left merge on gvkey
ccm1 = pd.merge(comp[['gvkey', 'datadate', 'be', 'count']], ccm, how = 'left', on = ['gvkey'])
ccm1['yearend'] = ccm1['datadate'] + YearEnd(0)
# create 'jdate' for further merge with crsp dataset
ccm1['jdate'] = ccm1['yearend'] + MonthEnd(6)

In [41]:
# set link date bounds
ccm2 = ccm1[(ccm1['jdate'] >= ccm1['linkdt']) & (ccm1['jdate'] <= ccm1['linkenddt'])]
ccm2 = ccm2[['gvkey', 'permno', 'datadate', 'yearend', 'jdate', 'be', 'count']]

In [42]:
# link comp and crsp
ccm_jun=pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])

# Finally, calculate book to market ratio
ccm_jun['beme']=ccm_jun['be']*1000/ccm_jun['dec_me']

In [43]:
ccm_jun.info()
ccm_jun.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277192 entries, 0 to 277191
Data columns (total 18 columns):
permno      277192 non-null int64
date        277192 non-null datetime64[ns]
jdate       277192 non-null datetime64[ns]
shrcd       277192 non-null int64
exchcd      277192 non-null int64
retadj      277192 non-null float64
me          277192 non-null float64
wt          264142 non-null float64
cumretx     276384 non-null float64
mebase      264927 non-null float64
lme         277192 non-null float64
dec_me      277192 non-null float64
gvkey       277192 non-null object
datadate    277192 non-null datetime64[ns]
yearend     277192 non-null datetime64[ns]
be          244439 non-null float64
count       277192 non-null int64
beme        244439 non-null float64
dtypes: datetime64[ns](4), float64(9), int64(4), object(1)
memory usage: 40.2+ MB


,permno,date,jdate,shrcd,exchcd,retadj,me,wt,cumretx,mebase,lme,dec_me,gvkey,datadate,yearend,be,count,beme
0,10001,1987-06-30,1987-06-30,11,3,0.051429,5822.125,5602.187450,0.959184,6033.125,5636.3125,6937.00,012994,1986-06-30,1986-12-31,7.037,0,1.014415
1,10001,1988-06-30,1988-06-30,11,3,-0.012039,6200.000,6379.562514,1.063830,5822.125,6386.0000,5828.00,012994,1987-06-30,1987-12-31,7.038,1,1.207618
2,10001,1989-06-30,1989-06-30,11,3,0.017143,7007.000,6944.000017,1.120000,6200.000,6986.0000,6362.25,012994,1988-06-30,1988-12-31,7.286,2,1.145192
3,10001,1990-06-29,1990-06-30,11,3,0.014103,10052.250,9759.750037,1.392857,7007.000,10013.2500,10347.75,012994,1989-06-30,1989-12-31,8.466,3,0.818149
4,10001,1991-06-28,1991-06-30,11,3,0.078481,11266.500,10181.124978,1.076923,10052.250,10408.2500,10013.00,012994,1990-06-30,1990-12-31,9.438,4,0.942575


### NYSE stock bucket breakdown

In [46]:
# select NYSE stocks for bucket breakdown
# exchcd = 1 and positive beme and positive me and shrcd in (10,11) and at least 2 years in comp
nyse_cond = (ccm_jun['exchcd'] == 1) & (ccm_jun['beme'] > 0) & (ccm_jun['me'] > 0) & (ccm_jun['count'] >= 1) & ((ccm_jun['shrcd'] == 10) | (ccm_jun['shrcd'] == 11))
# NOTE: & --> AND; | --> OR
                
nyse=ccm_jun[nyse_cond]

# size breakdown
# to_frame() convert a Pandas Series to a Pandas DataFrame
nyse_sz=nyse.groupby(['jdate'])['me'].median().to_frame().reset_index().rename(columns={'me':'sizemedn'})

# beme breakdown
# the following we use describe() to use its percetiles, HOW convinient!
nyse_bm=nyse.groupby(['jdate'])['beme'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_bm=nyse_bm[['jdate','30%','70%']].rename(columns={'30%':'bm30', '70%':'bm70'})

# merge two bucket breakdown datasets
nyse_breaks = pd.merge(nyse_sz, nyse_bm, how='inner', on=['jdate'])

# merge back to our main dataset
ccm1_jun = pd.merge(ccm_jun, nyse_breaks, how='left', on=['jdate'])

In [47]:
ccm1_jun.head()

,permno,date,jdate,shrcd,exchcd,retadj,me,wt,cumretx,mebase,...,dec_me,gvkey,datadate,yearend,be,count,beme,sizemedn,bm30,bm70
0,10001,1987-06-30,1987-06-30,11,3,0.051429,5822.125,5602.187450,0.959184,6033.125,...,6937.00,012994,1986-06-30,1986-12-31,7.037,0,1.014415,542542.875,0.523415,0.947148
1,10001,1988-06-30,1988-06-30,11,3,-0.012039,6200.000,6379.562514,1.063830,5822.125,...,5828.00,012994,1987-06-30,1987-12-31,7.038,1,1.207618,496282.500,0.620128,1.118397
2,10001,1989-06-30,1989-06-30,11,3,0.017143,7007.000,6944.000017,1.120000,6200.000,...,6362.25,012994,1988-06-30,1988-12-31,7.286,2,1.145192,560041.125,0.574367,1.006842
3,10001,1990-06-29,1990-06-30,11,3,0.014103,10052.250,9759.750037,1.392857,7007.000,...,10347.75,012994,1989-06-30,1989-12-31,8.466,3,0.818149,555233.250,0.491551,0.934872
4,10001,1991-06-28,1991-06-30,11,3,0.078481,11266.500,10181.124978,1.076923,10052.250,...,10013.00,012994,1990-06-30,1990-12-31,9.438,4,0.942575,540153.750,0.621685,1.214132


### Functions for assigning characteristics bucket

In [48]:
# functions
def sz_bucket(row):
    if row['me']==np.nan:
        value=''
    elif row['me']<=row['sizemedn']:
        value='S'
    else:
        value='B'
    return value

def bm_bucket(row):
    if 0<=row['beme']<=row['bm30']:
        value = 'L'
    elif row['beme']<=row['bm70']:
        value='M'
    elif row['beme']>row['bm70']:
        value='H'
    else:
        value=''
    return value

### Assign size and btm portfolios

In [49]:
# assign size portfolio
ccm1_jun['szport'] = np.where((ccm1_jun['beme']>0) & 
                            (ccm1_jun['me']>0) & 
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(sz_bucket, axis=1), '')

# assign book-to-market portfolio
ccm1_jun['bmport'] = np.where((ccm1_jun['beme']>0) & 
                            (ccm1_jun['me']>0) & 
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(bm_bucket, axis=1), '')

# create positivebmeme and nonmissport variable
ccm1_jun['posbm'] = np.where((ccm1_jun['beme']>0) & 
                           (ccm1_jun['me']>0) & 
                           (ccm1_jun['count']>=1), 
                           1, 0)
ccm1_jun['nonmissport']=np.where((ccm1_jun['bmport']!=''), 1, 0)

# store portfolio assignment as of June
june = ccm1_jun[['permno','date', 'jdate', 'bmport','szport','posbm','nonmissport']]
june['ffyear'] = june['jdate'].dt.year

/usr/local/miniconda3/envs/minipy/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Merge with monthly returns

In [50]:
# merge back with monthly records
crsp3 = crsp3[['date','permno','shrcd','exchcd','retadj',
               'me','wt','cumretx','ffyear','jdate']]
ccm3=pd.merge(crsp3, 
              june[['permno','ffyear','szport','bmport','posbm','nonmissport']], 
              how='left', 
              on=['permno','ffyear'])

In [51]:
# keeping only records that meet the criteria
ccm4=ccm3[(ccm3['wt']>0) & 
          (ccm3['posbm']==1) &
          (ccm3['nonmissport']==1) & 
          ((ccm3['shrcd']==10) | (ccm3['shrcd']==11))]

### Value-weighted returns

In [52]:
# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError: # in case of zero division
        return np.nan

In [57]:
# value-weigthed return
# https://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
# weighted-average function suggested by Wes McKinney
vwret = ccm4.groupby(['jdate','szport','bmport']).apply(wavg, 'retadj', 'wt').to_frame().reset_index().rename(columns={0: 'vwret'})
vwret['sbport'] = vwret['szport'] + vwret['bmport'] # <= concat string

# firm count
vwret_n = ccm4.groupby(['jdate','szport','bmport'])['retadj'].count().reset_index().rename(columns={'retadj':'n_firms'})
vwret_n['sbport'] = vwret_n['szport'] + vwret_n['bmport']

# preliminary results
# really is transposing the data: use date as index, columns is sbport, values is weighted average
ff_factors = vwret.pivot(index='jdate', columns='sbport', values='vwret').reset_index()
ff_nfirms = vwret_n.pivot(index='jdate', columns='sbport', values='n_firms').reset_index()

### Create SMB and HML factors

In [61]:
# create SMB and HML factors
ff_factors['WH']=(ff_factors['BH']+ff_factors['SH'])/2
ff_factors['WL']=(ff_factors['BL']+ff_factors['SL'])/2
ff_factors['WHML'] = ff_factors['WH']-ff_factors['WL']

ff_factors['WB']=(ff_factors['BL']+ff_factors['BM']+ff_factors['BH'])/3
ff_factors['WS']=(ff_factors['SL']+ff_factors['SM']+ff_factors['SH'])/3
ff_factors['WSMB'] = ff_factors['WS']-ff_factors['WB']
ff_factors=ff_factors.rename(columns={'jdate':'date'})

# n firm count
ff_nfirms['H']=ff_nfirms['SH']+ff_nfirms['BH']
ff_nfirms['L']=ff_nfirms['SL']+ff_nfirms['BL']
ff_nfirms['HML']=ff_nfirms['H']+ff_nfirms['L']

ff_nfirms['B']=ff_nfirms['BL']+ff_nfirms['BM']+ff_nfirms['BH']
ff_nfirms['S']=ff_nfirms['SL']+ff_nfirms['SM']+ff_nfirms['SH']
ff_nfirms['SMB']=ff_nfirms['B']+ff_nfirms['S']
ff_nfirms['TOTAL']=ff_nfirms['SMB']

ff_nfirms=ff_nfirms.rename(columns={'jdate':'date'})

### Compare with FF

In [64]:
# download data from wrds
_ff = conn.get_table(library='ff', table='factors_monthly')
_ff=_ff[['date','smb','hml']]
_ff['date']=_ff['date']+MonthEnd(0)

# correlation between our created FF factors and published FF
_ffcomp = pd.merge(_ff, ff_factors[['date','WSMB','WHML']], how='inner', on=['date'])
_ffcomp70=_ffcomp[_ffcomp['date']>='01/01/1970']
print(stats.pearsonr(_ffcomp70['smb'], _ffcomp70['WSMB']))
print(stats.pearsonr(_ffcomp70['hml'], _ffcomp70['WHML']))

(0.9960860785871981, 0.0)
(0.9803002075590305, 0.0)
